# AUV 2D

In [ ]:
import sys
sys.path.append("../src")
from imperium.dynamics.symbolic import System
from sympy import *
init_printing()

In [ ]:
# state
x, y, vx, vy = Matrix(symbols('x y v_x v_y', real=True))
state = Matrix([x, y, vx, vy])

# controls
ut, ux, uy = symbols('u_t', real=True, nonnegative=True), *symbols('u_x u_y', real=True)
control = Matrix([ut, ux, uy])

# constants
CD, m, A, T, rho = symbols('C_D m A T rho', real=True, nonnegative=True)

# state dynamics
dynamics = Matrix([
    vx,
    vy,
    T*ut*ux/m,
    T*ut*uy/m
])

# homotopy parameter
alpha = symbols('alpha', real=True, nonnegative=True)

# Lagrangian
lagrangian = alpha*ut + (1 - alpha)*ut**2

# equality constraints
equality = Matrix([sqrt(ux**2 + uy**2) - 1])

# inequality constraints
inequality = Matrix([ut-1, -ut, ux-1, -1-ux, uy-1, -1-uy])

In [ ]:
# instantiate the system
sys = System(state, control, dynamics, lagrangian)
sys.dfs

In [ ]:
# optimal thrust direction (solved by hand with geometric method)
utheta = -Matrix([sys.l[2], sys.l[3]])/sqrt(sys.l[2]**2 + sys.l[3]**2)
utheta.T

In [ ]:
# gradient of Hamiltonian
sys.KKT([(ux, utheta[0]), (uy, utheta[1])])
sys.KKTeqs

In [ ]:
# solving
sol = solve(sys.KKTeqs, sys.KKTvars, simplify=True, force=True)

In [ ]:
# now assign optimal controls
sys.uo = Matrix([[sol[ut]], utheta])
sys.uo.T

In [ ]:
# make some fortran code!
sys.codegen("../src/imperium/dynamics/auv2d", "F")

# Using fortran code

In [1]:
import sys
sys.path.append("../src")
from imperium.dynamics import auv2d
from imperium.segment import Indirect
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# instantiate segment
seg = Indirect(auv2d, (10, 10))

In [3]:
# set boundries
seg.set(0, np.array([0, 0, 0, 0]), 100, np.array([10, 10, 0, 0]))

In [15]:
fs0 = np.hstack((seg.s0, np.random.random(len(seg.s0))))
lbl = [0, -1, -1]
ubl = [1, 1, 1]
ubounded = np.array([min(max(u, lb), ub) for u, lb, ub in zip(seg.dynamics.control(*fs0, *seg.alpha, 0).flatten(), lbl, ubl)])
ubounded

array([ 0.30636595, -0.71496078, -0.69916456])

In [ ]:
plt.plot(seg.times, seg.states[:, 5])
plt.show()

In [ ]:
np.hstack((seg.s0, np.array([1, 1, 1, 1])))